## Code Objectives
* Get the SSH and velocity data for each pass during the science phase in the area of interest
* Compute the mean of the variables for each pass
* Plot the different variables

### Import packages

In [3]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import plots
import matplotlib.patches as patches
import functions
import glob
import os
import matplotlib.colors as mcolors
from shapely.geometry import LineString, Point
import geopandas as gpd 
from matplotlib.animation import FuncAnimation

### Get and process the data

In [7]:
output_dir = "/Users/dihyachaal/Desktop/DataShelf/PassGroups/"

pass_dirs = [d for d in glob.glob(os.path.join(output_dir, "*")) if os.path.isdir(d)]

pass_files = {}
Data = []
Data_array = []
means_ssh = []
means_ugos = []
means_vgos = []
longitude = []
latitude = []
times = []

for p_dir in pass_dirs:
    pass_num = os.path.basename(p_dir)
    files = sorted(glob.glob(os.path.join(p_dir, "*.nc")))
    pass_files[pass_num] = files
    datasets = [xr.open_dataset(fp) for fp in files]
    datasets = [functions.drop_num_nadir(ds) for ds in datasets]
    
    combined = xr.concat(datasets, dim='file') 
    ugos_mean = combined['ugos_filtered'].mean(dim='file', skipna=True)
    vgos_mean = combined['vgos_filtered'].mean(dim='file', skipna=True)
    ssha_mean = combined['ssha_filtered'].mean(dim='file', skipna=True)
    mdt_mean = combined['mdt'].mean(dim='file', skipna=True)
    time = combined['time'].isel(num_lines=0)
    ssh = combined['mdt']+combined['ssha_unfiltered']
    mean_ssh = ssh.mean(dim='file', skipna=True)
    lat = datasets[0]['latitude']
    lon = datasets[0]['longitude']
    
    
    Data.append(datasets)
    Data_array.append(combined)
    means_ssh.append(mean_ssh)
    means_ugos.append(ugos_mean)
    means_vgos.append(vgos_mean)
    longitude.append(lon)
    latitude.append(lat)
    times.append(time)
    
len(times)

19

In [8]:
bathy_data = 'GEBCO_14_May_2025_b03991ce039e/gebco_2024_n75.0_s42.0_w-80.0_e-40.0.nc'
bathy = xr.open_dataset(bathy_data) 

In [9]:
longitude_all = xr.concat(longitude, dim="track")
latitude_all = xr.concat(latitude, dim="track")
means_ugeos = xr.concat(means_ugos, dim="track")
means_vgeos = xr.concat(means_vgos, dim="track")
means_ssh = xr.concat(means_ssh, dim="track")

means_ugeos = means_ugeos.where((means_ugeos.values <= 1) & (means_ugeos.values >= -1))
means_vgeos = means_vgeos.where((means_vgeos.values <= 1) & (means_vgeos.values >= -1))

In [10]:
Line_strings = [LineString([(-55.6508, 53.3127),
                    (-53.8696, 53.8646)]), 
         LineString([ (-53.8696, 53.8646), 
                    (-50.2795,46.7089)]),
          LineString([(-50.2795,46.7089),
                    (-51.8235,45.8096)]), 
           LineString([(-51.8235,45.8096),
                   (-53.1607,46.5873)]) ]

In [ ]:
fig, ax, rect = plots.figure(figsize=(5, 5),nrows=1,ncols=1,region=[-65, -45, 43, 63])

plots.axis_shape(ax, rect, addCoastline=True, addCheckerboard=False)
ax.tick_params(right=False, labelright=False, top=False, labeltop=False)

plot_kwargs = dict(x="longitude", y="latitude", cmap="RdBu_r", transform=ccrs.PlateCarree(), vmin=-0.5, vmax=0.1)
cb_args_off = dict(add_colorbar=False)


step = 10  
for i in range(len(longitude_all.track)):
    ssh_i = means_ssh.isel(track=i)
    ugos_i = means_ugeos.isel(track=i)
    vgos_i = means_vgeos.isel(track=i)
    longitude_i = longitude_all.isel(track=i)
    latitude_i = latitude_all.isel(track=i)

    p = ssh_i.plot.pcolormesh(ax=ax, **plot_kwargs, **cb_args_off, zorder=1)
    ax.quiver(
        longitude_i[::step, ::step].values,
        latitude_i[::step, ::step].values,
        ugos_i[::step, ::step].values,
        vgos_i[::step, ::step].values,
        transform=ccrs.PlateCarree(),
        scale=20,
        color='k',
        zorder=2
    )
cbar = plt.colorbar(p, ax=ax, shrink=0.5, pad=0.015, orientation="vertical")
cbar.set_label("SSH [m]")


x=[-55.03333, -55.16233, -55.43833, -55.56367]
y=[53.53333,  53.491, 53.40833, 53.37517]
lcc = ax.plot(x,y, 'g', label='LCC moorings', transform=ccrs.PlateCarree(), zorder=3)[0]

mo1 = ax.plot(-53.00083, 50.74967, 'o', c='b', label='SWOT1-01', markersize=3, transform=ccrs.PlateCarree(), zorder=3)[0]  # position of mooring 1
mo2 = ax.plot(-53.26417, 50.16467, 'o', c='fuchsia', label='SWOT2-01', markersize=3, transform=ccrs.PlateCarree(), zorder=3)[0]  # position of mooring 2
mo3 = ax.plot(-52.58427, 49.74992, 'o', c='cyan', label='SWOT3-01', markersize=3, transform=ccrs.PlateCarree(), zorder=3)[0]  # position of mooring 3


ax.legend(handles=[lcc,mo1, mo2,mo3], fontsize=5, loc = 'upper right', bbox_to_anchor=(1.08, 0.98))

BC = (-1*bathy.elevation).plot.contour(ax=ax, transform=ccrs.PlateCarree(), colors='k', alpha=0.6, levels=np.arange(1000, 4500, 500), 
                                       linewidths=0.25, zorder=3)

BC2 = (-1*bathy.elevation).plot.contour(ax=ax, transform=ccrs.PlateCarree(), colors='k', alpha=0.6, levels=np.arange(300, 600, 100), 
                                       linewidths=0.25, zorder=3)

ax.text(-62, 52.5, "Labrador",
        transform=ccrs.PlateCarree(),
        horizontalalignment='center', zorder=4,
        fontsize=6)
ax.text(-56, 48.6, "Newfoundland",transform=ccrs.PlateCarree(), zorder=4, horizontalalignment='center', fontsize=4)
ax.text(-50.2, 46, "Grand",transform=ccrs.PlateCarree(),
        horizontalalignment='center', fontsize=4, zorder=3, c='white', fontweight='bold')
ax.text(-50.2, 45.8, "Banks",transform=ccrs.PlateCarree(),
        horizontalalignment='center', fontsize=4, zorder=3, c='white', fontweight='bold')

gdf = gpd.GeoDataFrame(geometry= Line_strings)

gdf.plot(ax=ax, color='darkviolet', linewidth=1, transform=ccrs.PlateCarree(), zorder=4)

lon_range = -53, -51
lat_range = 48, 50
square = patches.Rectangle((lon_range[0], lat_range[0]), lon_range[1]-lon_range[0], lat_range[1]-lat_range[0], edgecolor='white', linewidth=0.7,linestyle='--',
                           facecolor='none', transform=ccrs.PlateCarree(), zorder=3)
ax.add_patch(square)

plt.show() 
# fig.savefig("LScurrents.png", dpi=300)  

In [17]:
means_ugeos = means_ugeos.where((means_ugeos.values <= 0.5) & (means_ugeos.values >= -0.5))
means_vgeos = means_vgeos.where((means_vgeos.values <= 0.5) & (means_vgeos.values >= -0.5))

In [ ]:
fig, ax, rect = plots.figure(figsize=(3, 3),nrows=1,ncols=1,region=[-56, -50, 45, 54])
plots.axis_shape(ax, rect, addCoastline=True, addCheckerboard=False)

# cb_args = dict(add_colorbar=True, cbar_kwargs={"shrink": 0.5, "pad": 0.015, "label": "SSH [m]","orientation":"vertical"})
plot_kwargs = dict(x="longitude", y="latitude", cmap="RdBu_r", transform=ccrs.PlateCarree(), vmin=-0.5, vmax=0.1)
cb_args_off = dict(add_colorbar=False)

step = 6
for i in range(len(longitude_all.track)):
    ssh_i = means_ssh.isel(track=i)
    ugos_i = means_ugeos.isel(track=i)
    vgos_i = means_vgeos.isel(track=i)
    longitude_i = longitude_all.isel(track=i)
    latitude_i = latitude_all.isel(track=i)

    p = ssh_i.plot.pcolormesh(ax=ax, **plot_kwargs, **cb_args_off, zorder=1)
    ax.quiver(
        longitude_i[::step, ::step].values,
        latitude_i[::step, ::step].values,
        ugos_i[::step, ::step].values,
        vgos_i[::step, ::step].values,
        transform=ccrs.PlateCarree(),
        scale = 6,
        color='k',
        zorder=2
    )
# cbar = plt.colorbar(p, ax=ax, shrink=0.5, pad=0.015, orientation="vertical")
# cbar.set_label("SSH [m]")

ax.text(-55, 48.6, "Newfoundland",transform=ccrs.PlateCarree(), horizontalalignment='center', fontsize=3)
x=[-55.03333, -55.16233, -55.43833, -55.56367]
y=[53.53333,  53.491, 53.40833, 53.37517]
lcc = ax.plot(x,y, 'g', label='LCC moorings', transform=ccrs.PlateCarree(), zorder=3)[0]

mo1 = ax.plot(-53.00083, 50.74967, 'o', c='b', label='SWOT1-01', markersize=3, transform=ccrs.PlateCarree(), zorder=3)[0]  
mo2 = ax.plot(-53.26417, 50.16467, 'o', c='fuchsia', label='SWOT2-01', markersize=3, transform=ccrs.PlateCarree(), zorder=3)[0]  
mo3 = ax.plot(-52.58427, 49.74992, 'o', c='cyan', label='SWOT3-01', markersize=3, transform=ccrs.PlateCarree(), zorder=3)[0] 

ax.add_patch(square)

gdf.plot(ax=ax, color='darkviolet', linewidth=1, transform=ccrs.PlateCarree(), zorder=4)

plt.show() 
# fig.savefig("LScurrents_zoompolygon.png", dpi=300)  

KeyboardInterrupt: 

In [ ]:
# fig, ax, rect = plots.figure(figsize=(3, 3),nrows=1,ncols=1,region=[-54, -49, 45, 48])
# # fig, ax, rect = plots.figure(figsize=(5, 5),nrows=1,ncols=1,region=[-60, -50, 48, 60])
# plots.axis_shape(ax, rect, addCoastline=True, addCheckerboard=False)

# # cb_args = dict(add_colorbar=True, cbar_kwargs={"shrink": 0.5, "pad": 0.015, "label": "SSH [m]","orientation":"vertical"})
# plot_kwargs = dict(x="longitude", y="latitude", cmap="RdBu_r", transform=ccrs.PlateCarree(), vmin=-0.5, vmax=0.1)
# cb_args_off = dict(add_colorbar=False)

# step = 4
# for i in range(len(longitude_all.track)):
#     ssh_i = means_ssh.isel(track=i)
#     ugos_i = means_ugeos.isel(track=i)
#     vgos_i = means_vgeos.isel(track=i)
    
#     longitude_i = longitude_all.isel(track=i)
#     latitude_i = latitude_all.isel(track=i)

#     p = ssh_i.plot.pcolormesh(ax=ax, **plot_kwargs, **cb_args_off, zorder=3)
#     ax.quiver(
#         longitude_i[::step, ::step].values,
#         latitude_i[::step, ::step].values,
#         ugos_i[::step, ::step].values,
#         vgos_i[::step, ::step].values,
#         transform=ccrs.PlateCarree(),
#         scale = 6,
#         color='k',
#         zorder=4
#     )
# # cbar = plt.colorbar(p, ax=ax, shrink=0.5, pad=0.015, orientation="vertical")
# # cbar.set_label("SSH [m]")



# # norm = mcolors.PowerNorm(gamma=0.35, vmin=10, vmax=3500)
# # BP = (-1*bathy.elevation).plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), norm=norm, cmap='Blues', add_colorbar=False)

# BC = (-1*bathy.elevation).plot.contour(ax=ax, transform=ccrs.PlateCarree(), colors='k', alpha=0.6, levels=np.arange(1000, 4500, 500), 
#                                        linewidths=0.25, zorder=3)

# BC2 = (-1*bathy.elevation).plot.contour(ax=ax, transform=ccrs.PlateCarree(), colors='k', alpha=0.6, levels=np.arange(300, 600, 100), 
#                                        linewidths=0.25, zorder=3)
# #(200, 800, 200)
# # ax.clabel(BC, inline=True, fontsize=6, zorder=2)
# # ax.clabel(BC2, inline=True, fontsize=6, zorder=2)


# # geom = line_strings + points
# gdf = gpd.GeoDataFrame(geometry= Line_strings)

# gdf.plot(ax=ax, color='darkviolet', linewidth=1, transform=ccrs.PlateCarree(), zorder=4)

# plt.show() 
# # fig.savefig("LScurrents_zoomAvalonpng", dpi=300)  